In [3]:
MODEL_GPT = 'gpt-4o-mini'

In [5]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [7]:
import os
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv

# Şehir bilgisi input ile alınıyor
city = input("Hangi şehir için rehber oluşturulsun? ").strip().title()

# Wikipedia'dan bilgi çek
def get_wikipedia_intro(city):
    url = f"https://en.wikipedia.org/wiki/{city.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.select("p")
    intro = ""
    for p in paragraphs:
        if p.text.strip():
            intro += p.text.strip()
        if len(intro) > 600:
            break
    return intro

# Wikivoyage'dan bilgi çek
def get_wikivoyage_content(city):
    url = f"https://en.wikivoyage.org/wiki/{city.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.select("p")
    content = ""
    for p in paragraphs:
        content += p.text.strip() + "\n"
        if len(content) > 800:
            break
    return content

# (Opsiyonel) TripAdvisor - basit örnek
def get_tripadvisor_stub(city):
    return f"Visit https://www.tripadvisor.com/Search?q=things+to+do+in+{city.replace(' ', '+')}"

# Verileri çek
wiki = get_wikipedia_intro(city)
voyage = get_wikivoyage_content(city)
trip_link = get_tripadvisor_stub(city)

combined_info = f"""
Wikipedia Bilgisi:\n{wiki}

Wikivoyage Bilgisi:\n{voyage}

TripAdvisor bağlantısı: {trip_link}
"""

# GPT ile rehber oluştur
def generate_guide(city, info, model="gpt-4o-mini"):
    prompt = f"""
Aşağıdaki bilgilerden yararlanarak {city} şehri için 3 günlük bir turizm rehberi oluştur:
- Şehir tanıtımı
- Gezilecek yerler (5+)
- Ne yenir?
- 3 günlük öneri planı
- Pratik bilgiler

Bilgiler:
{info}
"""
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Sen deneyimli bir turizm rehberi yazarı ve seyahat planlayıcısısın."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1500
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Rehber üret
generate_guide(city, combined_info)

Hangi şehir için rehber oluşturulsun?  istanbul 


### İstanbul 3 Günlük Turizm Rehberi

#### Şehir Tanıtımı
İstanbul, Türkiye'nin en büyük ve en kalabalık şehri olup, hem Avrupa hem de Asya kıtalarını birleştiren tarihi bir köprü işlevi görmektedir. M.Ö. 7. yüzyılda Yunanlılar tarafından kurulan şehir, zamanla Roma ve Bizans İmparatorluklarına ev sahipliği yapmış, 1930 yılında resmi olarak İstanbul adını almıştır. Zengin tarihi, kültürel mirası ve büyüleyici manzaralarıyla İstanbul, hem turistler hem de yerli halk için keşfedilmeyi bekleyen bir hazine gibidir.

#### Gezilecek Yerler
1. **Ayasofya**: Bizans döneminin en önemli yapılarından biri olan Ayasofya, muazzam mimarisi ve tarihiyle mutlaka görülmeli.
2. **Sultanahmet Camii (Mavi Camii)**: Mavi çinileriyle ünlü bu cami, İstanbul’un simgelerinden biridir.
3. **Topkapı Sarayı**: Osmanlı İmparatorluğu'nun yönetim merkezi olan bu saray, etkileyici koleksiyonlarıyla ziyaretçilerini büyülüyor.
4. **Kapalıçarşı**: Dünyanın en büyük kapalı pazarlarından biri olan Kapalıçarşı, alışveriş yapmak için harika bir yerdir.
5. **Galata Kulesi**: İstanbul’un panoramik manzarasını görmek için ideal bir nokta.
6. **Bosphorus Turu**: İstanbul Boğazı’nda yapılan bir tekne turu, hem Avrupa hem de Asya kıyılarını görmek için harika bir fırsattır.

#### Ne Yenir?
- **Döner Kebap**: İstanbul’un olmazsa olmaz lezzetlerinden biri, mutlaka tadılmalı.
- **Meze**: Zengin çeşitleriyle meze kültürü, özellikle rakı sofralarının vazgeçilmezidir.
- **Balık Ekmek**: Eminönü’nde taze balık ekmek yemek, yerel bir deneyimdir.
- **Baklava**: Tatlı severler için, İstanbul baklavası deneyimlenmesi gereken bir tatlıdır.
- **Simit**: Şehirde sokaklarda satılan simit, kahvaltı için mükemmel bir atıştırmalıktır.

#### 3 Günlük Öneri Planı

**1. Gün: Tarihi Yarımada**
- Sabah: Ayasofya’yı ziyaret edin.
- Öğle: Sultanahmet Camii’ni gezip çevresindeki kafelerde öğle yemeği.
- Öğleden sonra: Topkapı Sarayı’nı keşfedin.
- Akşam: Kapalıçarşı’da alışveriş yapın ve yerel bir restoranda akşam yemeği.

**2. Gün: Boğaziçi ve Galata**
- Sabah: Boğaz’da tekne turu yapın.
- Öğle: Ortaköy’de kumpir ya da waffle yiyin.
- Öğleden sonra: Galata Kulesi’ni ziyaret edin ve çevresindeki kafelerde çay molası verin.
- Akşam: Karaköy’de bir balık restoranında akşam yemeği.

**3. Gün: Asya Yakası ve Modern İstanbul**
- Sabah: Kadıköy’deki pazarı gezip simit ve çay ile kahvaltı yapın.
- Öğle: Moda sahilinde yürüyüş yapın ve yerel restoranlarda öğle yemeği.
- Öğleden sonra: Üsküdar’a geçin ve Kız Kulesi’ni ziyaret edin.
- Akşam: Boğaz manzaralı bir restoranda akşam yemeği ile gezinizi sonlandırın.

#### Pratik Bilgiler
- **Ulaşım**: İstanbul'da toplu taşıma oldukça yaygındır. Metro, tramvay ve otobüsler ile şehri kolayca gezebilirsiniz. İstanbulkart edinerek daha ekonomik seyahat edebilirsiniz.
- **Hava Durumu**: Seyahat tarihinize göre hava durumunu kontrol edin, kışın soğuk ve yağışlı, yazın ise sıcak ve nemli olabilir.
- **Dil**: Türkçe resmi dil olmasına rağmen, birçok turistik bölgede İngilizce de yaygın olarak konuşulmaktadır.
- **Güvenlik**: İstanbul genel olarak güvenli bir şehir olsa da, kalabalık yerlerde dikkatli olmak her zaman önemlidir.
- **Para Birimi**: Türkiye’nin para birimi Türk Lirası (TRY)dır. Kredi kartları birçok yerde kabul edilir.

İstanbul'un zengin kültürünü ve tarihi dokusunu keşfetmek için bu planı takip edebilir, unutulmaz bir deneyim yaşayabilirsiniz. İyi yolculuklar!

In [9]:
# Required libraries
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
from openai import OpenAI

# Load API key from .env
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# Choose the model
MODEL = "gpt-4o-mini"

# Get user input
city = input("Which city should the travel guide be for? ").strip().title()

# Get intro from Wikipedia
def get_wikipedia_intro(city):
    url = f"https://en.wikipedia.org/wiki/{city.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.select("p")
    intro = ""
    for p in paragraphs:
        if p.text.strip():
            intro += p.text.strip()
        if len(intro) > 600:
            break
    return intro

# Get content from Wikivoyage
def get_wikivoyage_content(city):
    url = f"https://en.wikivoyage.org/wiki/{city.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.select("p")
    content = ""
    for p in paragraphs:
        content += p.text.strip() + "\n"
        if len(content) > 800:
            break
    return content

# TripAdvisor stub link
def get_tripadvisor_stub(city):
    return f"Visit https://www.tripadvisor.com/Search?q=things+to+do+in+{city.replace(' ', '+')}"

# Collect data
wiki = get_wikipedia_intro(city)
voyage = get_wikivoyage_content(city)
trip_link = get_tripadvisor_stub(city)

combined_info = f"""
Wikipedia Info:\n{wiki}

Wikivoyage Info:\n{voyage}

TripAdvisor link: {trip_link}
"""

# Generate guide using GPT
def generate_guide(city, info, model="gpt-4o-mini"):
    prompt = f"""
Using the information below, create a 3-day travel guide for the city of {city}:
- City introduction
- Must-see attractions (5 or more)
- Local food recommendations
- Suggested 3-day itinerary
- Practical tips

Information:
{info}
"""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an experienced travel guide writer and trip planner."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1500
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Generate and display guide
generate_guide(city, combined_info)


Which city should the travel guide be for?  lizbon


# 3-Day Travel Guide to Lisbon, Portugal

## City Introduction
Lisbon, the capital of Portugal, is a vibrant city known for its rich history, stunning architecture, and lively culture. Nestled on the banks of the Tagus River, Lisbon offers visitors a mix of traditional charm and modern flair. With its picturesque neighborhoods, historic tram rides, and delicious cuisine, the city is a feast for the senses. Whether you're wandering through the narrow streets of Alfama or enjoying a sunset view from the Miradouro de Santa Catarina, Lisbon promises an unforgettable experience.

## Must-See Attractions

1. **Belém Tower (Torre de Belém)**  
   A UNESCO World Heritage Site, this iconic tower was built in the 16th century to defend the city. Its stunning Manueline architecture and riverside location make it a must-visit.

2. **Jerónimos Monastery (Mosteiro dos Jerónimos)**  
   Another UNESCO site, this monastery is a masterpiece of Manueline architecture, showcasing intricate designs and a rich history tied to the Age of Discoveries.

3. **Alfama District**  
   The oldest neighborhood in Lisbon, Alfama is a labyrinth of narrow streets, traditional Fado music venues, and stunning viewpoints. It's the perfect place to get lost and discover local life.

4. **São Jorge Castle (Castelo de São Jorge)**  
   Perched on a hilltop, this historic castle offers panoramic views of the city. Explore its ancient walls and learn about Lisbon’s history through informative exhibits.

5. **Praça do Comércio**  
   This grand square located by the river is surrounded by impressive buildings. It’s a great spot to relax, enjoy a drink, and soak in the vibrant atmosphere.

6. **Lisbon Oceanarium**  
   One of the largest aquariums in Europe, the Oceanarium showcases marine life from various ocean habitats. It's a great family-friendly attraction.

## Local Food Recommendations

- **Bacalhau à Brás**: A traditional codfish dish made with eggs, onions, and fried potatoes.
- **Pastéis de Nata**: Delicious custard tarts that are a must-try when in Lisbon. Head to Pastéis de Belém for the original recipe.
- **Sardinhas Assadas**: Grilled sardines, especially popular during the summer festivals.
- **Francesinha**: A hearty sandwich filled with meats, covered in a spicy sauce, typically found in Lisbon's cafés.
- **Bifana**: A spicy pork sandwich served in a crusty roll, perfect for a quick bite.

## Suggested 3-Day Itinerary

### Day 1: Historic Lisbon
- **Morning**: Start with a visit to **Belém Tower** and **Jerónimos Monastery**. Enjoy a pastéis de nata at a local café in Belém.
- **Afternoon**: Explore **Praça do Comércio** and take a leisurely stroll along the waterfront.
- **Evening**: Head to the **Alfama District** for dinner. Try bacalhau à Brás at a local restaurant and enjoy Fado music.

### Day 2: Culture and Views
- **Morning**: Visit **São Jorge Castle** for breathtaking views of the city. Explore the castle grounds and learn about its history.
- **Afternoon**: Walk through the narrow streets of **Alfama**, visiting local shops and cafés. Stop for lunch and try a bifana.
- **Evening**: Head to **Miradouro de Santa Catarina** for sunset views. Enjoy dinner at a nearby restaurant featuring fresh seafood.

### Day 3: Modern Lisbon
- **Morning**: Visit the **Lisbon Oceanarium** to see its remarkable exhibits. 
- **Afternoon**: Spend time in the **Parque das Nações** area, where you can enjoy modern architecture and attractions. Have lunch here.
- **Evening**: Explore the vibrant streets of the **Bairro Alto** neighborhood. Enjoy tapas and drinks at a rooftop bar.

## Practical Tips
- **Getting Around**: Lisbon has an extensive public transport system, including trams, buses, and metro. Consider purchasing a Lisboa Card for unlimited travel and discounts on attractions.
- **Language**: Portuguese is the official language, but many locals speak English, especially in tourist areas.
- **Safety**: Lisbon is generally safe, but be mindful of pickpockets in crowded areas.
- **Best Time to Visit**: Spring (March to June) and fall (September to November) are the best times to visit, offering mild weather and fewer tourists.
- **Currency**: The currency used is the Euro (€). Make sure to carry some cash for small purchases, though credit cards are widely accepted.

Enjoy your trip to Lisbon, where history, culture, and cuisine await!